In [1]:
import torch
from torch import nn
import lightnet as ln
from torchinfo import summary
from model import model_builder
import functools

/home/gustavo/workstation/depth_estimation/codes/rgbd-yolov2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# model = ln.models.Darknet19(1000)
# model.load('weights/darknet19_448.weights')

# # Save as PyTorch weight file (Not strictly necessary, but it is faster than darknet weight files)
# model.save('weights/darknet19_448.pt')

# # Converting Darknet19 weights to Yolo (This is the same as the darknet19_448.conv.23.weights from darknet)
# model.save('weights/yolo-pretrained_darknet.pt', remap=ln.models.YoloV2.remap_darknet19)

# # Load yolo weights (Requires `strict=False`, because not all layers have weights in this file)
# detection_model = ln.models.YoloV2(3)
# detection_model.load('weights/yolo-pretrained_darknet.pt', strict=False)

## Depth Only Network

In [2]:
rgb_model = model_builder(num_classes=3, model_type="rgb")
summary(model=rgb_model, 
        input_size=(32, 3, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

/home/gustavo/workstation/depth_estimation/codes/rgbd-yolov2/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
YoloV2 (YoloV2)                               [32, 3, 416, 416]    [32, 40, 13, 13]     --                   True
├─FeatureExtractor (backbone)                 [32, 3, 416, 416]    [32, 1024, 13, 13]   --                   True
│    └─Sequential (module)                    [32, 3, 416, 416]    [32, 1024, 13, 13]   --                   True
│    │    └─Conv2dBatchAct (1_convbatch)      [32, 3, 416, 416]    [32, 32, 416, 416]   928                  True
│    │    └─MaxPool2d (2_max)                 [32, 32, 416, 416]   [32, 32, 208, 208]   --                   --
│    │    └─Conv2dBatchAct (3_convbatch)      [32, 32, 208, 208]   [32, 64, 208, 208]   18,560               True
│    │    └─MaxPool2d (4_max)                 [32, 64, 208, 208]   [32, 64, 104, 104]   --                   --
│    │    └─Conv2dBatchAct (5_convbatch)      [32, 64, 104, 104]   [32, 128, 104, 104] 

In [19]:
depth_model = model_builder(num_classes=3, model_type="depth")

# Print a summary using torchinfo (uncomment for actual output)
summary(model=depth_model, 
        input_size=(32, 1, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

/home/gustavo/workstation/depth_estimation/codes/rgbd-yolov2/.venv/lib/python3.10/site-packages/lightnet/network/module/_lightnet.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental 

Replacing first layer with new conv: Conv2dBatchAct(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), LeakyReLU(negative_slope=0.1, inplace=True))


Layer (type (var_name))                       Input Shape          Output Shape         Param #              Trainable
YoloV2 (YoloV2)                               [32, 1, 416, 416]    [32, 40, 13, 13]     --                   True
├─FeatureExtractor (backbone)                 [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    └─Sequential (module)                    [32, 1, 416, 416]    [32, 1024, 13, 13]   --                   True
│    │    └─Conv2dBatchAct (1_convbatch)      [32, 1, 416, 416]    [32, 32, 416, 416]   352                  True
│    │    └─MaxPool2d (2_max)                 [32, 32, 416, 416]   [32, 32, 208, 208]   --                   --
│    │    └─Conv2dBatchAct (3_convbatch)      [32, 32, 208, 208]   [32, 64, 208, 208]   18,560               True
│    │    └─MaxPool2d (4_max)                 [32, 64, 208, 208]   [32, 64, 104, 104]   --                   --
│    │    └─Conv2dBatchAct (5_convbatch)      [32, 64, 104, 104]   [32, 128, 104, 104] 

In [23]:
depth_pretrained = torch.load("models/depth_99.pth")

/tmp/ipykernel_13368/3341212066.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  depth_pretrained = torch.load("models/depth_99.pth")


In [25]:
list(depth_pretrained.keys())

['backbone.module.1_convbatch.layers.0.weight',
 'backbone.module.1_convbatch.layers.1.weight',
 'backbone.module.1_convbatch.layers.1.bias',
 'backbone.module.1_convbatch.layers.1.running_mean',
 'backbone.module.1_convbatch.layers.1.running_var',
 'backbone.module.1_convbatch.layers.1.num_batches_tracked',
 'backbone.module.3_convbatch.layers.0.weight',
 'backbone.module.3_convbatch.layers.1.weight',
 'backbone.module.3_convbatch.layers.1.bias',
 'backbone.module.3_convbatch.layers.1.running_mean',
 'backbone.module.3_convbatch.layers.1.running_var',
 'backbone.module.3_convbatch.layers.1.num_batches_tracked',
 'backbone.module.5_convbatch.layers.0.weight',
 'backbone.module.5_convbatch.layers.1.weight',
 'backbone.module.5_convbatch.layers.1.bias',
 'backbone.module.5_convbatch.layers.1.running_mean',
 'backbone.module.5_convbatch.layers.1.running_var',
 'backbone.module.5_convbatch.layers.1.num_batches_tracked',
 'backbone.module.6_convbatch.layers.0.weight',
 'backbone.module.6_co

In [2]:
fusion_model = model_builder(num_classes=3, model_type="rgbd", fuse_layer=15)

/home/gustavo/workstation/depth_estimation/codes/rgbd-yolov2/model.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rgb_state_dict = torch.load("models/rgb_state_dict.pt

Weights equivalent


In [3]:
# Print a summary using torchinfo (uncomment for actual output)
summary(model=fusion_model, 
        input_size=(32, 4, 416, 416), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Conv2dBatchAct: 3, Conv2d: 5, BatchNorm2d: 5, LeakyReLU: 5, Conv2dBatchAct: 3, Conv2d: 5, BatchNorm2d: 5, LeakyReLU: 5]

In [33]:
input_tensor = torch.randn(32, 4, 416, 416)
output_tensor = fusion_model(input_tensor)
print(output_tensor.shape)

torch.Size([32, 40, 13, 13])


In [11]:
fusion_weight = fusion_model.state_dict()['layers.1.regular.11_convbatch.layers.1.weight'].cpu()
rgb_weight = rgb_model.state_dict()['backbone.module.11_convbatch.layers.1.weight'].cpu()

In [10]:
torch.equal(fusion_weight, rgb_weight)

True